<a href="https://colab.research.google.com/github/dorinhazan/FinalProject-DataScience/blob/main/classification_grouping_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install openai
# !pip install --upgrade openai
# !pip install striprtf --quiet
# !pip install striprtf


In [ ]:
import openai
import zipfile
import json
import os
import pandas as pd
from openai import OpenAI
from striprtf.striprtf import rtf_to_text

In [ ]:
with open("token_api_final_proj.rtf", "r", encoding="utf-8") as f:
    rtf = f.read()

key = rtf_to_text(rtf).strip()
os.environ["OPENAI_API_KEY"] = key

print("OPENAI_API_KEY loaded:", bool(os.getenv("OPENAI_API_KEY")))

OPENAI_API_KEY loaded: True


In [ ]:
results_ranking_path = "merged_ranking.txt"
# `ranking_res_zip` taken from git in path "Results/ranking_results_and_zip"
ranking_res_zip = "/content/ranking_results.zip"

with zipfile.ZipFile(ranking_res_zip, 'r') as z, open(results_ranking_path, 'w', encoding='utf-8') as out:
    for info in z.infolist():
        if info.filename.endswith('.txt'):
            out.write(f"===== {info.filename} =====\n")
            with z.open(info) as f:
                for raw in f:
                    out.write(raw.decode('utf-8', errors='replace'))
            out.write("\n\n")

print(f"Done — all .txt merged into `{results_ranking_path}`")


FileNotFoundError: [Errno 2] No such file or directory: '/content/ranking_results.zip'

In [ ]:
obs_res_zip = '/content/json_results.zip'
extract_dir = '/content/extracted_jsons'
merged_json_path = 'merged_obs_extraction.json'

with zipfile.ZipFile(obs_res_zip, 'r') as zf:
    zf.extractall(extract_dir)

# Merge into one dictionary, extending duplicates’ observables
merged_report_results = {}
for filename in os.listdir(extract_dir):
    # only process .json files
    if not filename.lower().endswith('.json'):
        continue

    full_path = os.path.join(extract_dir, filename)
    with open(full_path, 'r') as f:
        data = json.load(f)

    for md_name, doc in data.items():
        if md_name not in merged_report_results:
            merged_report_results[md_name] = doc
        else:
            # extend observables list from both
            merged_report_results[md_name]["observables"].extend(doc.get("observables", []))

with open(merged_json_path, 'w') as f:
    json.dump(merged_report_results, f, indent=2)

Create Classification Groups

In [ ]:
# Need to load `merged_obs_extraction.json` that contain all the observables extracted from observable_extraction_assistant_and_observable_rank_validator.ipynb
merged_json_path = 'merged_obs_extraction.json'
with open(merged_json_path, 'r', encoding='utf-8') as f:
    merged_obs_extraction = json.load(f)

In [ ]:
# Extract, lowercase, and dedupe classification values
unique_classifications = {
    obs['classification'].lower()
    for doc in merged_obs_extraction.values()
    for obs in doc.get('observables', [])
    if 'classification' in obs
}

In [ ]:
# Classification value taken from procedure description table
unique_classifications.update(['api',
 'api call',
 'api function',
 'api hook',
 'archive file',
 'attack effect',
 'attack technique',
 'authentication credential',
 'authentication technique',
 'boot record',
 'c2 server',
 'circuit breaker',
 'code snippet',
 'command',
 'command line argument',
 'command message',
 'command parameters',
 'command sequence',
 'command value',
 'command/utility',
 'communication channel',
 'communication interface',
 'communication port',
 'communication priority level',
 'communication protocol',
 'config/data file',
 'configuration data',
 'configuration file',
 'configuration parameter',
 'credential',
 'credential list',
 'data',
 'data block',
 'data manipulation',
 'data transfer',
 'database server',
 'device',
 'device configuration value',
 'device state',
 'device status parameter',
 'distributed control system',
 'dll file',
 'document',
 'domain name',
 'driver file',
 'dynamic-link library',
 'electrical device',
 'email message',
 'error state',
 'event log',
 'executable command',
 'executable file',
 'exploit payload',
 'exploit technique',
 'facility',
 'file',
 'file type',
 'file/script',
 'firewall rule',
 'firewall utility',
 'firmware',
 'frequency converter drive',
 'function block memory location',
 'hardware component',
 'hex value',
 'hmi device',
 'host device',
 'http request',
 'ics address',
 'ics application',
 'ics command',
 'ics command message',
 'ics communication payload',
 'ics communication protocol',
 'ics configuration',
 'ics configuration file',
 'ics controller',
 'ics data',
 'ics data block recording',
 'ics device',
 'ics device parameter',
 'ics field',
 'ics function block',
 'ics gateway',
 'ics i/o snapshot',
 'ics network communication',
 'ics object',
 'ics parameter',
 'ics payload component',
 'ics process data',
 'ics project file',
 'ics protocol',
 'ics protocol field',
 'ics protocol module',
 'ics protocol parameter',
 'ics protocol value',
 'ics software module',
 'ics systems',
 'ics tag',
 'identifier constant',
 'impact condition',
 'industrial communication protocol',
 'industrial control device',
 'industrial control module',
 'industrial control system',
 'industrial control system (ics) device',
 'industrial control system device',
 'industrial control system platform',
 'industrial controller (cpu)',
 'industrial network bus',
 'integrity check routine',
 'log files',
 'malicious file',
 'malicious ics function block',
 'malicious logic',
 'malicious software',
 'malware',
 'malware module',
 'manufacturing facility',
 'medical product',
 'memory access pattern',
 'memory access request',
 'memory write attempt',
 'module',
 'monitor',
 'network communication',
 'network communication technique',
 'network configuration',
 'network connection',
 'network data',
 'network device',
 'network entity',
 'network identifier',
 'network packet',
 'network port',
 'network protocol',
 'network segment',
 'network service',
 'network service/port',
 'network traffic',
 'network traffic event',
 'opc server',
 'opc ua address space',
 'opc ua server',
 'operating system api',
 'operational mode',
 'os system call',
 'packaging tool',
 'payload file',
 'plc',
 'plc block',
 'plc code',
 'plc code component',
 'plc cpu',
 'plc data block',
 'plc data block content (magic value)',
 'plc device',
 'plc function block',
 'plc i/o image',
 'plc operating system',
 'plc organization block',
 'plc program organization unit',
 'plc runtime constraint',
 'process',
 'process name list',
 'process termination',
 'profibus communication module',
 'program transfer operation',
 'project file',
 'protective relay',
 'protocol',
 'protocol field',
 'protocol object data',
 'protocol version',
 'proxy server',
 'python function call',
 'registry key',
 'remote access service',
 'remote terminal unit',
 'removable media',
 'resource identifier',
 'safety instrumented system',
 'script',
 'script engine/interpreter',
 'script file',
 'script framework',
 'sensor reading',
 'serial port',
 'server',
 'server information',
 'service control',
 'service/process',
 'servo drive',
 'shellcode',
 'signature value',
 'software/program',
 'software/tool',
 'software/tool (malware)',
 'sql command',
 'sql stored procedure',
 'storage device',
 'string',
 'url',
 'user account',
 'variable state',
 'vulnerability',
 'vulnerability identifier',
 'web script',
 'windows registry key'])

In [ ]:
# gross classification values from the mitre attack site as the group name
gross_classifications = set([
"System",
"Module",
"Hardware",
"Device",
"PLC",
"POU",
"Identifier",
"Information",
"Credential",
"Data",
"Command",
"API",
"Code",
"Process",
"Configuration",
"Log",
"File",
"Payload",
"Software",
"Service",
"Firmware",
"Server",
"Communication",
"Protocol",
"Protocol Field",
"Connection Port",
"Vulnerability",
"Attack Technique"])

In [61]:
unique_classifications('security organization')

TypeError: 'set' object is not callable

In [78]:
# … your existing imports …
import openai
import json
import os

# add:
import concurrent.futures

# … your code that sets up:
#   unique_classifications  = set([...])    # lowercased strings
#   gross_classifications   = set([...])    # your 28 MITRE labels

# Make ordered lists so our prompts are stable
sorted_uniques = sorted(unique_classifications)
sorted_gross   = sorted(gross_classifications)

# 1) Decide on a chunk size small enough to fit in a single prompt.
#    ~100–200 per chunk is usually safe.
chunk_size = 100
chunks = [
    sorted_uniques[i : i + chunk_size]
    for i in range(0, len(sorted_uniques), chunk_size)
]

def group_chunk(chunk):
    """Send one chunk to GPT and return its JSON mapping."""
    prompt = f"""
You are a cybersecurity analyst expert. Your job is to group a list of observable “classification” labels under gross classifications from MITRE ATT&CK® Enterprise, or if not matched to any existing gross classification - create a new category that matches the observable group.

---
### Input
   - A set of unique observable classifications:
     {chunk}
   - A set of gross classifications from MITRE ATT&CK® Enterprise:
     {sorted_gross}

---
### Task
   - For each unique observable classification, choose exactly one gross classification it best fits.
   - If it does not clearly belong to any of the gross classification labels given, assign it to a new category of your own choosing, appending “– new” to that category name (e.g. “Data Management – new”).
   - If you’re really unsure, use a the name of the unique observable classification as the new category, appending “– new” to that category name.

---
### Output
   - Return **only** a single JSON object (no extra text).
   - Each key must be either one of the 28 gross classification names or one of your “– new” categories.
   - Each value must be an array of the unique observable classifications assigned to that key.

### MANDATORY
  - Every single unique observable classification **must** appear in the output—there can be **no** omissions.
  - If in doubt, create a new category named as the  unique observable classification value.
  - You can not use Unknown or similar as a new category.



"""
    client = OpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
    )

    response = client.chat.completions.create(
            model="o3-mini-2025-01-31",
            messages=[{"role": "user", "content": prompt}]
        )
    raw = response.choices[0].message.content.strip()
    print(raw)
    # strip off any leading/trailing chatter so we get pure JSON
    start = raw.find("{")
    end   = raw.rfind("}")
    json_str = raw[start : end+1]
    return json.loads(json_str)


# 2) Fire all chunks in parallel (max 5 threads at once to be polite to the API)
all_partial = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as pool:
    futures = [pool.submit(group_chunk, c) for c in chunks]
    for fut in concurrent.futures.as_completed(futures):
        all_partial.append(fut.result())

# 3) Merge all the sub‐mappings into one big map
full_mapping = {}
for part in all_partial:
    for gross, subs in part.items():
        full_mapping.setdefault(gross, []).extend(subs)

# 4) Deduplicate each list
for gross in full_mapping:
    full_mapping[gross] = sorted(set(full_mapping[gross]))

# 5) Save it out
with open("full_groups_mapping.json", "w", encoding="utf-8") as f:
    json.dump(full_mapping, f, indent=2, ensure_ascii=False)

print(f"✅ Done! {len(full_mapping)} gross‐classes, {sum(len(v) for v in full_mapping.values())} total mappings written to full_groups_mapping.json")


{
  "API": [
    "api",
    "api call",
    "api command",
    "api function",
    "api hook",
    "api interface",
    "api method"
  ],
  "Attack Technique": [
    "access vector",
    "attack pattern",
    "attack technique",
    "attack vector",
    "backdoor"
  ],
  "Code": [
    "assembly code",
    "assembly code/disassembly",
    "assembly instruction",
    "autolisp code",
    "autolisp script",
    "batch script",
    "binary code snippet",
    "binary function",
    "c runtime function",
    "c struct definition"
  ],
  "Command": [
    "batch command",
    "batch file command"
  ],
  "Communication": [
    "anomalous network connection",
    "anonymity network",
    "anonymizing network"
  ],
  "Configuration": [
    "access control",
    "access control mechanism",
    "access mode",
    "access protection table",
    "authentication mechanism"
  ],
  "Credential": [
    "account",
    "authentication credential"
  ],
  "Data": [
    "application data",
    "base64 encoded

In [79]:
# 1) get a flat set of everything you actually assigned:
assigned = {
    item.strip().lower()
    for vals in full_mapping.values()
    for item in vals
}

# 2) normalize your original uniques too:
normalized_uniques = { u.strip().lower() for u in unique_classifications }

# 3a) to check “did we map every unique into some gross category?”:
missing = normalized_uniques - assigned
if missing:
    print("❌ These unique labels were never assigned:", missing)
else:
    print("✅ All unique labels have been assigned!")

# 3b) if you instead want to check “are all my unique labels themselves valid gross_classifications?”:
missing_in_gross = normalized_uniques - { g.strip().lower() for g in gross_classifications }
if missing_in_gross:
    print("⚠️ These uniques aren’t in your gross list:", missing_in_gross)
else:
    print("✅ Every unique label is one of the gross classifications!")


❌ These unique labels were never assigned: {'disassembly data', 'industrial protection relay', 'http post request format', 'malware/worm', 'malware/tool', 'industrial control system (ics) product'}
⚠️ These uniques aren’t in your gross list: {'mp field value', 'physical address', 'constant', 'generic placeholder (nm parameter)', 'remote access protocol', 'ics component – exhaust valve', 'hex dump (decompressed record)', 'vulnerability bulletin', 'ics attack framework', 'ransomware', 'decoded user key', 'volume serial number', 'ics process data', 'industrial network bus', 'industrial control component - valve', 'program structure description', 'database file', 'dynamic link library (dll)', 'industrial communication protocol', 'assembly code/disassembly', 'encryption key', 'directory', 'domain fragment', 'industrial control equipment', 'script engine/interpreter', 'email authentication protocol', 'http parameter', 'twitter handle', 'ip address range (geographic)', 'account', 'ui command'

In [80]:
# Build reverse map: sub-classification (lowercase) → gross group name
reverse_map = {}
for gross_group, sub_list in full_mapping.items():
    for sub in sub_list:
        reverse_map[sub.lower()] = gross_group

Rename column and extract only the relavent for comprison

In [81]:
# Iterate over each file and its observables, renaming keys as requested
for filename, content in merged_obs_extraction.items():
    for obs in content.get('observables', []):
        if 'observable_rank' in obs:
            obs['artifact_details'] = obs.pop('observable_rank')
        if 'classification' in obs:
            obs['fine_classification'] = obs.pop('classification')

In [82]:
for doc_name, doc_data in merged_obs_extraction.items():
    for obs in doc_data.get('observables', []):
        cls = obs.get('fine_classification', '').lower()
        # look up the group, default to 'Unknown' if not found
        obs['gross_classification'] = reverse_map.get(cls, 'Unknown')

out_path = '/content/merged_with_gross_classification-reports.json'
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(merged_obs_extraction, f, indent=2, ensure_ascii=False)

print(f"Done—saved annotated file to {out_path}")

Done—saved annotated file to /content/merged_with_gross_classification-reports.json


Convert csv table into json object in order to compare between observables and add `gross_classification` col

In [83]:
# Need to upload the `Procedure_descriptions_observables` csv - load it from this link https://docs.google.com/spreadsheets/d/1dLCtedM3K46sLqIcTHv8IFORr55mLGboQKdXdPsjerg/edit?gid=0#gid=0
csv_path = "/content/updated_Procedure_descriptions_observables_using_o3_mini_prompt_with_deprecation.csv"
observables_porc_desc_df = pd.read_csv(csv_path)

observables_porc_desc_df = observables_porc_desc_df.rename(columns={"classification": "fine_classification"})

wanted = [
    "observable_value",
    "artifact_details",
    "data_source",
    "fine_classification",
    "notes",
    "description",
    "STIX_supported",
    "proprietary_artifact",
    "parser",
]

# Reindex (fills any missing with None)
observables_porc_desc_df = observables_porc_desc_df.reindex(columns=wanted, fill_value=None)

# Drop duplicates on observable_value
observables_porc_desc_df = observables_porc_desc_df.drop_duplicates(subset=['observable_value'], keep='first')

# Add gross_classification via vectorized map
observables_porc_desc_df['gross_classification'] = (
    observables_porc_desc_df['fine_classification']
      .str.lower()
      .map(reverse_map)
      .fillna('Unknown')
)

observables_porc_desc_df.set_index("observable_value") \
    .to_json(
       "observables_by_value.json",
       orient="index",
       indent=2,
       force_ascii=False
    )

out_path = '/content/merged_with_gross_classification-table.json'
records = observables_porc_desc_df.to_dict(orient='records')
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Saved annotated file to {out_path}")


Saved annotated file to /content/merged_with_gross_classification-table.json
